In [1]:
import google.generativeai as genai
from google.generativeai.types import ContentType
from PIL import Image
from IPython.display import Markdown
import time
import cv2

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
GOOGLE_API_KEY = 'REMOVED'
genai.configure(api_key=GOOGLE_API_KEY)

In [12]:
from PIL import Image

def convert_webp_to_jpg(input_path, output_path):
    # Open the WebP image
    with Image.open(input_path) as img:
        # Convert and save the image as JPEG
        img.convert("RGB").save(output_path, "JPEG")
        
        
convert_webp_to_jpg('image1.webp', 'image1.jpeg')

In [20]:
from PIL import Image

def convert_webp_to_png(input_path, output_path):
    # Open the WebP image
    with Image.open(input_path) as img:
        # Save the image as PNG
        img.save(output_path, "PNG")

# Example usage
input_webp = "image1.webp"
output_png = "output_image.png"
convert_webp_to_png(input_webp, output_png)
print(f"Converted {input_webp} to {output_png}")


Converted image1.webp to output_image.png


In [61]:
image = genai.upload_file('output_image.png')

In [62]:
image

In [67]:
model = genai.GenerativeModel(model_name="models/gemini-1.5-flash")

prompt =f"""
This image contains prescrition of a doctor, please extract the following information:
After you get the necessary information parse it in json format and return it.
Only return the json data with no additional text
keys of the json should be: 'medication_details'
medication_details should be a list of dictionaries.
some commong abrevations in the prescription are:
1. OD: Once a day
2. BD: Twice a day
3. TDS: Thrice a day
4. QID: Four times a day
5. HS: At bedtime
6. PRN: As needed
7. AC: Before meals
9. D : day
meidcation_details should have keys: 'name': string, 'dose':string, frequency_days:number, 'timings':list of strings"""

safe = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]
    
response = model.generate_content([image, prompt],safety_settings=safe)

# print(response.text)

In [68]:
print(response.text)

```json
{
"medication_details": [
{
"name": "CALPOL",
"dose": "4 ML",
"frequency_days": 3,
"timings": [
"Q6H"
]
},
{
"name": "DELCON",
"dose": "3 ML",
"frequency_days": 5,
"timings": [
"TDS"
]
},
{
"name": "LEVOUIN",
"dose": "3 ML",
"frequency_days": 5,
"timings": [
"TDS"
]
},
{
"name": "MEFTAL-P",
"dose": "3 ML",
"frequency_days": null,
"timings": [
"SOS"
]
}
]
}
```


In [72]:
def get_medicine_details_from_persription(image_path):
    convert_webp_to_png(image_path, 'output_image.png')
    image = genai.upload_file('output_image.png')
    model = genai.GenerativeModel(model_name="models/gemini-1.5-flash")
    prompt =f"""
    This image contains prescrition of a doctor, please extract the following information:
    After you get the necessary information parse it in json format and return it.
    Only return the json data with no additional text
    keys of the json should be: 'medication_details'
    medication_details should be a list of dictionaries.
    some commong abrevations in the prescription are:
    1. OD: Once a day
    2. BD: Twice a day
    3. TDS: Thrice a day
    4. QID: Four times a day
    5. HS: At bedtime
    6. PRN: As needed
    7. AC: Before meals
    9. D : day
    10 QNH: every n hours a day
    meidcation_details should have keys: 'name': string, 'dose':string, frequency_days:number, 'timings':list of strings"""
    safe = [
        {
            "category": "HARM_CATEGORY_DANGEROUS",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_NONE",
        },
    ]
    response = model.generate_content([image, prompt],safety_settings=safe)
    return response.text

In [73]:
print(get_medicine_details_from_persription('image1.webp'))

```json
{
"medication_details": [
{
"name": "CALPOL",
"dose": "4 ML",
"frequency_days": 3,
"timings": [
"Q6H"
]
},
{
"name": "DELCON",
"dose": "3 ML",
"frequency_days": 5,
"timings": [
"TDS"
]
},
{
"name": "LEVOUIN",
"dose": "3 ML",
"frequency_days": 5,
"timings": [
"TDS"
]
},
{
"name": "MEFTAL-P",
"dose": "3 ML",
"frequency_days": null,
"timings": [
"SOS"
]
}
]
}
```
